# Missing Values Exercises


## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:


```python
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}
```

### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.


## Exercises

### Exercise 1

Today, we will be using the ACS data we used during out first `pandas` exercise to examine the US income distribution, and how it varies by race. Note that because the US income distribution has a very small number of people with *extremely* high incomes, and the ACS is just a sample of Americans, the far right tail of the distribution will not be very well estimated. However, this data should suffice for helping to understand wealth inequality in the United States. 

To begin, load the ACS Data we used in our first pandas exercise. That [data can be found here](https://github.com/nickeubank/MIDS_Data/tree/master/US_AmericanCommunitySurvey). We'll be working with `US_ACS_2017_10pct_sample.dta`. 

In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

df = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

### Exercise 2

Let's begin by calculating the median US incomes from this data (recall that income is stored in the `inctot` variable). Store the answer in `results` under the key `"ex2_avg_income"`.

In [2]:
results = {}
results["ex2_avg_income"] = df["inctot"].mean()
# Print the results of results["ex2_avg_income"]
print(f"Average income: ${results['ex2_avg_income']:,}")

Average income: $1,723,646.2703978634


### Exercise 3

Hmmm... That doesn't look right. The average American is definitely not earning that much a year! Let's look at the values of `inctot` using `value_counts()`. Do you see a problem?

Now use `value_counts()` with the argument `normalize=True` to see proportions of the sample that report each value instead of the count of people in each category. What percentage of our sample has an income of 9,999,999? Store that proportion (between 0 and 1) as `"ex3_share_making_9999999"`. What percentage has an income of 0? Store that proportion as `"ex3_share_making_zero"`.

(Recall `.value_counts()` returns a Series, so you can pull values out with our usual pandas tools.)

In [3]:
df["inctot"].value_counts(normalize=True)
results["ex3_share_making_9999999"] = df["inctot"].value_counts(normalize=True)[9999999]
results["ex3_share_making_zero"] = df["inctot"].value_counts(normalize=True)[0]
# Print the results of results["ex3_share_making_9999999"] and results["ex3_share_making_zero"]
print(
    f"Percentage of shares making $9999999: {results['ex3_share_making_9999999']:.2f}"
)
print(f"Percentage of shares making $0: {results['ex3_share_making_zero']:.2f}")

Percentage of shares making $9999999: 0.17
Percentage of shares making $0: 0.11


### Exercise 4

As we discussed before, the ACS uses a value of 9999999 to denote that income information is not available for someone. The problem with using this kind of "sentinel value" is that pandas doesn't understand that this is supposed to denote missing data, and so when it averages the variable, it doesn't know to ignore 9999999. 

To help out `pandas`, use the `replace` command to replace all values of 9999999 with `np.nan`. 

In [4]:
import numpy as np

# wrong code
# df["inctot"][df["inctot"] == 9999999].replace(np.nan)

df["inctot"] = df["inctot"].replace(9999999, np.nan, inplace=False)

### Exercise 5

Now that we've properly labeled our missing data as `np.nan`, let's calculate the average US income once more. Store the answer in `results` under the key `"ex5_avg_income"`.

In [5]:
average = df["inctot"].mean(skipna=True)
results["ex5_avg_income"] = average
# Print the results of results["ex5_avg_income"]
print(f"Average income: ${results['ex5_avg_income']:,}")

Average income: $40,890.177564946454


### Exercise 6

OK, now we've been able to get a reasonable average income number. As we can see, a major advantage of using `np.nan` is that `pandas` knows that `np.nan` observations should just be ignored when we are calculating means. 

But it's not enough to just get rid of the people who had `inctot` values of 9999999. We also need to know why those values were missing. Suppose, for example, that the value of 9999999 was used for anyone who made more than 100,000 dollars: if we just dropped those people, then our estimate of average income wouldn't mean much, would it?

So let's make sure we understand *why* data is missing for some people. If you recall from our last exercise, it seemed to be the case that most of the people who had incomes of 9999999 were children. Let's make sure that's true by looking at the distribution of the variable `age` for people for whom `inctot` is missing (i.e. subset the data to people with `inctot` missing, then look at the values of `age` with `value_counts()`).

Then do the opposite: look at the distribution of the `age` variable for people who whom `inctot` is *not* missing. 

Can you determine when 9999999 was being used? Is it ok we're excluding those people from our analysis?

Note: In this data, Python doesn't understand `age` is a number; it thinks it is a string because the original data has categories like "90 (90+ in 1980 and 1990)" and "less than 1 year old". So you can't just use `min()` or `max()`. We'll discuss converting string variables into numbers in a future class.

In [6]:
df["inctot"].value_counts()

inctot
0.0         33679
30000.0      4778
50000.0      4414
40000.0      4413
20000.0      4067
            ...  
246600.0        1
90810.0         1
341380.0        1
15790.0         1
505400.0        1
Name: count, Length: 8470, dtype: int64

In [7]:
df["age"][df["inctot"].isna()].value_counts()

age
10    3997
9     3977
14    3847
12    3845
13    3800
      ... 
39       0
38       0
37       0
36       0
96       0
Name: count, Length: 97, dtype: int64

In [8]:
df["age"][df["inctot"].notna()].value_counts()

age
60                      4950
54                      4821
59                      4776
56                      4776
58                      4734
                        ... 
5                          0
4                          0
3                          0
2                          0
less than 1 year old       0
Name: count, Length: 97, dtype: int64

After our analysis, we found that people with young 
9999999 was used for children, who do not have income. Since we are doing analysis on income, it is okay we drop these values.

### Exercise 7

Great, so now we know why those people had missing data, and we're ok with excluding them. 

But as we previously noted, there are also a lot of observations of zero income in our data, and it's not clear that we want everyone with a zero-income *should* be included in this average, since those may be people who are retired, or in school. 

Let's limit our attention to people who are currently working by subsetting to only employed respondents. We can do this using `empstat`. Remember you can use `value_counts()` to see what values of `empstat` are in the data!

In [9]:
df["empstat"].value_counts()
df = df[df["empstat"] == "employed"]

### Exercise 8

Now let's estimate the racial income gap in the United States. What is the average salary for employed Black Americans, and what is the average salary for employed White Americans? In percentage terms, how much more does the average White American make than the average Black American?

**Note:** these values are not quite accurate estimates. As we'll discuss in later lessons, to get completely accurate estimates from the ACS we have to take into account how people were selected to be interviewed. But you get pretty good estimates in most cases even without weights—your estimate of the racial wage gap without weights is within 5\% of the corrected value. 

**Note:** This is actually an underestimate of the wage gap. The US Census treats Hispanic respondents as a sub-category of "White." While all ethnic distinctions are socially constructed, and so on some level these distinctions are all deeply problematic, this coding is inconsistent with what most Americans think of when they hear the term "White," a term *most* Americans think of as a category that is mutually exclusive of being Hispanic or Latino (categories which are also usually conflated in American popular discussion). With that in mind, most researchers working with US Census data split "White" into "White, Hispanic" and "White, Non-Hispanic" using `race` *and* `hispan`. But for the moment, just identify "White" respondents using the value in `race`.

Store your results in `results` under the keys `"ex8_avg_income_black"`, `"ex8_avg_income_white"`, and the percentage difference as `ex8_racial_difference`. Please note the wording above when calculating the percentage difference to ensure you get the reference category correct, and interpret your result as well.

In [10]:
df["race"].value_counts()

race
white                               116017
black/african american/negro         13175
other asian or pacific islander       6424
other race, nec                       5755
two major races                       3135
chinese                               2149
american indian or alaska native      1290
three or more major races              426
japanese                               387
Name: count, dtype: int64

In [11]:
results["ex8_avg_income_black"] = df["inctot"][
    df["race"] == "black/african american/negro"
].mean()
results["ex8_avg_income_white"] = df["inctot"][df["race"] == "white"].mean()
results["ex8_racial_difference"] = (
    (results["ex8_avg_income_white"] - results["ex8_avg_income_black"])
    / results["ex8_avg_income_black"]
    * 100
)
# Print the results of results["ex8_avg_income_black"] and results["ex8_racial_difference"]
print("Average income for black people:", results["ex8_avg_income_black"])
print("Racial difference in average income:", results["ex8_racial_difference"])

Average income for black people: 41747.949905123336
Racial difference in average income: 44.85299006275197


### Exercise 9


As noted above, these estimates are not actually *quite* correct because we aren't using survey weights. To calculate a weighted average that takes into account survey weights, you need to use the following formula:

$$weighted\_mean\_of\_x = \frac{\sum_i x_i * weight_i}{\sum_i weight_i}$$

(As you can see, when $weight_i$ is constant for all observations, this just simplifies to our normal formula for mean values. It is only when weights vary across individuals that weights must be explicitly addressed).

In this data, weights are stored in the variable `perwt`, which is the number of people for which each observation is a stand-in (the inverse of that observations sampling probability). 

Using the formula, re-calculate the *weighted* average income for both populations and store them as `ex9_avg_income_white` and `ex9_avg_income_black`.


In [12]:
df_white = df[df["race"] == "white"]
df_black = df[df["race"] == "black/african american/negro"]
results["ex9_avg_income_white"] = (sum(df_white["perwt"] * df_white["inctot"])) / sum(
    df_white["perwt"]
)
results["ex9_avg_income_black"] = (sum(df_black["perwt"] * df_black["inctot"])) / sum(
    df_black["perwt"]
)
print("Average income for white people:", results["ex9_avg_income_white"])
print("Average income for black people:", results["ex9_avg_income_black"])

Average income for white people: 58361.48196061399
Average income for black people: 40430.953355310274


### Exercise 10

Now calculate the weighted average income gap between *non-Hispanic* White Americans and Black Americans. What percentage more do employed White non-Hispanic Americans earn than employed Black Americans? Store as `"ex10_wage_gap"`.

In [13]:
df["hispan"].value_counts()
df_white_nh = df_white[df_white["hispan"] == "not hispanic"]
mean_white_nh = (sum(df_white_nh["perwt"] * df_white_nh["inctot"])) / sum(
    df_white_nh["perwt"]
)
results["ex10_wage_gap"] = (
    (mean_white_nh - results["ex9_avg_income_black"])
    / results["ex9_avg_income_black"]
    * 100
)
# Print the results of results["ex10_wage_gap"]
print(f"Wage gap: {results['ex10_wage_gap']:.2f}%")

Wage gap: 52.53%


### Exercise 11

Is that greater or less than the difference you found in Exercise 8? Why do you think that's the case?

In [14]:
weighed_gap = (
    (results["ex9_avg_income_white"] - results["ex9_avg_income_black"])
    / results["ex9_avg_income_black"]
    * 100
)
# Print the results of results["ex10_wage_gap"]
print(f"Wage gap: {weighed_gap:.2f}%")

Wage gap: 44.35%


* We calculatd the wighted difference between white and black, and there is not much difference between it and the non-weighted value.
* So the increase in the gap is resutled by the dropping of hispanic population among white population. Compard to nonhispanic white people, hispanic people tends to earn less money.

In [15]:
assert set(results.keys()) == {
    "ex2_avg_income",
    "ex3_share_making_9999999",
    "ex3_share_making_zero",
    "ex5_avg_income",
    "ex8_avg_income_black",
    "ex8_avg_income_white",
    "ex8_racial_difference",
    "ex9_avg_income_black",
    "ex9_avg_income_white",
    "ex10_wage_gap",
}

In [16]:
results

{'ex2_avg_income': 1723646.2703978634,
 'ex3_share_making_9999999': 0.1689665333350052,
 'ex3_share_making_zero': 0.10557547867738336,
 'ex5_avg_income': 40890.177564946454,
 'ex8_avg_income_black': 41747.949905123336,
 'ex8_avg_income_white': 60473.15372747098,
 'ex8_racial_difference': 44.85299006275197,
 'ex9_avg_income_white': 58361.48196061399,
 'ex9_avg_income_black': 40430.953355310274,
 'ex10_wage_gap': 52.52989147705372}